In [80]:
#Bring in modules
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import pandas as pd

In [81]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [31]:
# Retrieve the parent divs for all the articles
results = soup.find_all(class_='list_text')
#headlines = []
#articles = []
news=[]

for result in results:        
        # Identify and return title of a news item
        title_div= result.find(class_='content_title')
        headline = title_div.find('a').text        
        #headlines.append(headline)
        
        # Identify and return paragraph of the news item
        article = result.find(class_='article_teaser_body').text
        #articles.append(news_paragraph)
        
        #Create dataframe        
        news.append({"title": headline, "article": article}) 
        print(news)

[{'title': 'NASA Social Media and Websites Win Webby Awards ', 'article': 'NASA\'s social media presence, the InSight mission social media accounts, NASA.gov and SolarSystem.NASA.gov will be honored at the 2019 Webby Awards - "the Oscars of the Internet."'}]
[{'title': 'NASA Social Media and Websites Win Webby Awards ', 'article': 'NASA\'s social media presence, the InSight mission social media accounts, NASA.gov and SolarSystem.NASA.gov will be honored at the 2019 Webby Awards - "the Oscars of the Internet."'}, {'title': "NASA's InSight Detects First Likely 'Quake' on Mars", 'article': 'While their causes are still unknown, one of three shaking events looks a lot like the quakes detected on the Moon by the Apollo missions.'}]
[{'title': 'NASA Social Media and Websites Win Webby Awards ', 'article': 'NASA\'s social media presence, the InSight mission social media accounts, NASA.gov and SolarSystem.NASA.gov will be honored at the 2019 Webby Awards - "the Oscars of the Internet."'}, {'ti

In [32]:
## Scrape and store Mars images from JPL ##
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [33]:
# Capture Feature area box image url
mars_pic = soup.find_all(class_="carousel_container")
for m in mars_pic:
    image_url = m.find('a')['data-fancybox-href']
featured_image_url = 'https://www.jpl.nasa.gov'+image_url
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19643_ip.jpg


In [48]:
# Scrape and store Mars Weather
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
# Retrieve page with the requests module
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')


In [68]:
#Find and store mars_weather
mars_weather = soup.find(class_ = 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
twitter_url = mars_weather.find("a").text
mars_weather = mars_weather.text.replace(twitter_url, "")
print(mars_weather)


InSight sol 144 (2019-04-23) low -98.7ºC (-145.7ºF) high -17.6ºC (0.4ºF)
winds from the SW at 4.2 m/s (9.5 mph) gusting to 11.1 m/s (24.8 mph)
pressure at 7.40 hPa


In [85]:
#Scrape Mars Facts 
url = 'https://space-facts.com/mars/'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
table = soup.find(class_="tablepress tablepress-id-mars")


['Equatorial Diameter:' 'Polar Diameter:' 'Mass:' 'Moons:'
 'Orbit Distance:' 'Orbit Period:' 'Surface Temperature:' 'First Record:'
 'Recorded By:']
['6,792 km' '6,752 km' '6.42 x 10^23 kg (10.7% Earth)'
 '2 (Phobos & Deimos)' '227,943,824 km (1.52 AU)' '687 days (1.9 years)'
 '-153 to 20 °C' '2nd millennium BC' 'Egyptian astronomers']


In [112]:
mars_data = pd.read_html(str(table))
headers = mars_data[0][0].values
facts = mars_data[0][1].values
mars_facts = list(zip(headers, facts))
#mars_facts = dict(mars_facts)
#mars_facts('headers','facts')

mars_facts


[('Equatorial Diameter:', '6,792 km'),
 ('Polar Diameter:', '6,752 km'),
 ('Mass:', '6.42 x 10^23 kg (10.7% Earth)'),
 ('Moons:', '2 (Phobos & Deimos)'),
 ('Orbit Distance:', '227,943,824 km (1.52 AU)'),
 ('Orbit Period:', '687 days (1.9 years)'),
 ('Surface Temperature:', '-153 to 20 °C'),
 ('First Record:', '2nd millennium BC'),
 ('Recorded By:', 'Egyptian astronomers')]

In [37]:
#Scrape images from Mars Hemispheres
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
hemispheres=[]


#Retrieve the hemisphere names
hem_names = soup.find_all(class_='description')

for h in hem_names:
    #find web address of high res image
    web_loc = h.find('a',class_="itemLink product-item" )['href']
    image_url = 'https://astrogeology.usgs.gov' + web_loc
    #Go to the page with the high res image and scrape the image
    browser.visit(image_url)
    html = browser.html
    soup = bs(html, 'html.parser')
    full= soup.find(class_='downloads')
    full_image = full.find('a')['href']
    #full_images.append(full_image)
    
    #find name of hemisphere   
    title = h.find('h3').text
    hemispheres.append({'title': title, 'image_url': full_image})
print(hemispheres)

[{'title': 'Cerberus Hemisphere Enhanced', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [38]:
#Create one python dictionary
mars = {"news":news, "picture":featured_image_url, "weather":mars_weather,
                   "facts": mars_facts, "hemispheres":hemispheres}
print(mars)

{'news': [{'title': 'NASA Social Media and Websites Win Webby Awards ', 'article': 'NASA\'s social media presence, the InSight mission social media accounts, NASA.gov and SolarSystem.NASA.gov will be honored at the 2019 Webby Awards - "the Oscars of the Internet."'}, {'title': "NASA's InSight Detects First Likely 'Quake' on Mars", 'article': 'While their causes are still unknown, one of three shaking events looks a lot like the quakes detected on the Moon by the Apollo missions.'}, {'title': "Things Are Stacking up for NASA's Mars 2020 Spacecraft", 'article': 'As the July 2020 launch date inches closer, the next spacecraft headed to the Red Planet is assembled for more testing.'}, {'title': "Curiosity Tastes First Sample in 'Clay-Bearing Unit'", 'article': 'This new region on Mars might reveal more about the role of water on Mount Sharp.'}, {'title': "More Testing for Mars InSight's 'Mole'", 'article': "After the mission's heat probe began hammering last week, it appears to have hit a 

In [39]:
browser.quit()